In [377]:
import pandas as pd
import numpy as np

In [378]:
data = pd.read_csv("DataScientist.csv",index_col="Unnamed: 0",parse_dates=[9])

In [379]:
data.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1
1,1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1
2,2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,True
3,3,Data Analyst,$111K-$181K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1
4,4,"Director, Data Science",$111K-$181K (Glassdoor est.),"Director, Data Science - (200537)\nDescription...",3.4,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1


## Data Cleaning Steps
    

##### Cleaning and creating features from salary

In [380]:
#drop nans in salary
data = data[data["Salary Estimate"]!="-1"]

In [381]:
data["Salary Estimate"] = data["Salary Estimate"].str.strip()

In [382]:
data["Salary Estimate"] = data["Salary Estimate"].apply(lambda x:x.split("(")[0])

In [383]:
import re
data["Salary Estimate"] = data["Salary Estimate"].apply(lambda x : re.sub("[$K]",'',x))

In [384]:
data["salary_hr"] = data["Salary Estimate"].apply(lambda x:1 if "per hour" in x.lower() else 0)

In [385]:
data["Salary Estimate"] = data["Salary Estimate"].apply(lambda x:re.sub("per hour",'',x.lower()))

In [386]:
min_sal,max_sal = data["Salary Estimate"].apply(lambda x:x.split("-")[0]),data["Salary Estimate"].apply(lambda x:x.split("-")[1])          

In [387]:
data["min_sal"] = min_sal
data["max_sal"] = max_sal

In [388]:
data.min_sal = data.min_sal.astype(np.int64)
data.max_sal = data.max_sal.astype(np.int64)

In [389]:
#creating avg-salary feature
data["avg_sal"] = (data.min_sal + data.max_sal) / 2

##### Clean Company Name feature  

In [390]:
#cleaning company names
data['Company'] = data['Company Name'].apply(lambda x:re.sub('(\n\d(\.)?(\d)+)+','', x))

In [391]:
# dropping as we don't need it
data.drop("Company Name",axis=1,inplace=True)

##### Extract state from Location

In [392]:
#getting only state from location
data['job_state'] = data.Location.apply(lambda x:x.split(',')[1])

##### Another feature could be added which tells if location of job is the same as the headquarters of the company.


In [393]:
data['job_in_hq'] = data.apply(lambda x:1 if x.Location.lower().strip() == x.Headquarters.lower().strip() else 0,axis=1)                                            

##### Getting age of company from Founded column

In [394]:
#creating age of company feature
data["age_of_company"] = data.Founded.apply(lambda x:2020-x if x>0 else x)

##### Renaming missing values to -1

In [395]:
data.Size = data.Size.replace({'Unknown':"-1"})

In [396]:
data['Type of ownership'] = data['Type of ownership'].replace({"Unknown":"-1"})

In [397]:
data['Industry'] = data['Industry'].replace({"Unknown":"-1"})

In [398]:
data['Revenue'] = data['Revenue'].replace({"Unknown / Non-Applicable":"-1"})

##### Creating Features from job-description

In [399]:
data['need_R'] = data['Job Description'].apply(lambda x:1 if 'R' in x else 0)

In [400]:
lst = ["python","pandas","keras","tensorflow","pytorch"]
data['need_Python'] = data['Job Description'].apply(lambda x: 1 if any(ele in x.lower() for ele in lst) else 0)               

In [401]:
lst = ["spark","pig","hadoop","hive","big data"]
data['big_data'] = data['Job Description'].apply(lambda x: 1 if any(ele in x.lower() for ele in lst) else 0)               

In [402]:
lst = ['tableau','power bi','powerbi','qlik','d3.js','d3']
data['visual_skills'] = data['Job Description'].apply(lambda x: 1 if any(ele in x.lower() for ele in lst) else 0)

In [403]:
data['need_sql'] = data['Job Description'].apply(lambda x: 1 if 'sql' in x.lower() else 0)        

In [404]:
lst = ['aws','docker','azure','gcp']
data['cloud_skills'] =  data['Job Description'].apply(lambda x: 1 if any(ele in x.lower() for ele in lst) else 0)

#### Data after cleaning and Feature Engineering

In [412]:
data.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Location,Headquarters,Size,Founded,Type of ownership,...,Company,job_state,job_in_hq,age_of_company,need_R,need_Python,big_data,visual_skills,need_sql,cloud_skills
0,0,Senior Data Scientist,111-181,"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,...,Hopper,NY,0,13,1,1,1,1,1,0
1,1,"Data Scientist, Product Analytics",111-181,"At Noom, we use scientifically proven methods ...",4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,...,Noom US,NY,1,12,0,1,0,0,1,0
2,2,Data Science Manager,111-181,Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,"New York, NY","New York, NY",1 to 50 employees,-1,-1,...,Decode_M,NY,1,-1,1,1,0,0,0,0
3,3,Data Analyst,111-181,Sapphire Digital seeks a dynamic and driven mi...,3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,...,Sapphire Digital,NJ,1,1,1,1,0,1,1,0
4,4,"Director, Data Science",111-181,"Director, Data Science - (200537)\nDescription...",3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,...,United Entertainment Group,NY,1,13,1,1,1,0,0,0


##### Output Cleaned Data

In [407]:
data.to_csv('DataScientist_cleaned.csv',index=False)

In [409]:
#just to check if we did it right!
# pd.read_csv('DataScientist_cleaned.csv')